This script processes large geospatial TIFF files to extract valid pixel values along with their geographic coordinates. Using rasterio, it reads TIFF data in memory-efficient chunks, replaces nodata values with NaN, and writes the latitude, longitude, and pixel values for non-NaN pixels to a CSV file. Designed for large datasets, it ensures efficient processing and outputs a CSV with valid geospatial data.

In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import imagecodecs
import csv
from rasterio.plot import show
from tifffile import imread
from rasterio.windows import Window
from io import BytesIO
import os

In [16]:
file_paths = [
    r"C:\Users\nolan\Downloads\MS_NDVI.tif"
    # Add all 24 file paths here
]

In [17]:
# Define a chunk size
chunk_size = 1000  # Adjust based on your system's memory

# Loop over each file in the list
for file_path in file_paths:
    # Initialize a counter for valid (non-NaN) pixels
    valid_pixel_count = 0

    # Check if the file exists before processing
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue  # Skip to the next file if not found

    # Extract the file name without extension to use for the CSV file and column name
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    csv_filename = f'{base_filename}.csv'

    # Open and read the TIFF file with rasterio using windows
    try:
        with rasterio.open(file_path) as dataset:
            print(f"TIFF file '{base_filename}' read successfully with rasterio!\n")
            
            # Get the total size of the image
            height, width = dataset.height, dataset.width
            print(f"Image dimensions: {height}x{width}\n")
            
            # Get the transform to map pixel coordinates to geographic coordinates
            transform = dataset.transform
            
            # Check for nodata value in the dataset
            nodata_value = dataset.nodata
            if nodata_value is None:
                print(f"No 'nodata' value found in file '{base_filename}'.")
            else:
                print(f"'Nodata' value in file '{base_filename}': {nodata_value}")
            
            # Open a CSV file to write the output for this file
            with open(csv_filename, mode='w', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                
                # Write the header (Latitude, Longitude, and File Name as the column header for pixel values)
                csv_writer.writerow(['Latitude', 'Longitude', base_filename])
                
                # Loop over the image in chunks (windows)
                for i in range(0, height, chunk_size):
                    for j in range(0, width, chunk_size):
                        # Define a window to read a chunk of the image
                        window = Window(j, i, min(chunk_size, width - j), min(chunk_size, height - i))
                        
                        # Read the chunk
                        image_chunk = dataset.read(1, window=window)

                        # Replace 'nodata' values with np.nan
                        if nodata_value is not None:
                            image_chunk = np.where(image_chunk == nodata_value, np.nan, image_chunk)

                        # Create a mask for valid (non-NaN) pixels
                        valid_mask = ~np.isnan(image_chunk)

                        # If there are no valid pixels in this chunk, skip the rest of the loop
                        if not np.any(valid_mask):
                            continue
                        
                        # Get the row and column indices of the valid pixels
                        rows, cols = np.where(valid_mask)
                        
                        # Convert the pixel coordinates to geographic coordinates
                        coords = [rasterio.transform.xy(transform, row + i, col + j) for row, col in zip(rows, cols)]
                        
                        # Extract the actual pixel values for valid pixels
                        pixel_values = image_chunk[valid_mask]
                        
                        # Write each valid coordinate and pixel value to the CSV file
                        for coord, pixel_value in zip(coords, pixel_values):
                            lat, lon = coord
                            csv_writer.writerow([lat, lon, pixel_value])
                            
                            # Increment the valid pixel counter
                            valid_pixel_count += 1

            print(f"\nData and coordinates have been saved to '{csv_filename}'.")
            print(f"Total number of non-NaN pixels processed and stored for file '{base_filename}': {valid_pixel_count}")

    except Exception as e:
        print(f"Error reading TIFF file {file_path} with rasterio: {e}")

TIFF file 'MS_NDVI' read successfully with rasterio!

Image dimensions: 6700x29850

'Nodata' value in file 'MS_NDVI': nan

Data and coordinates have been saved to 'MS_NDVI.csv'.
Total number of non-NaN pixels processed and stored for file 'MS_NDVI': 97667406
